In [ ]:
import numpy as np
import string
from queue import PriorityQueue


In [ ]:
with open("day12.txt", "r") as f:
    data = f.read()


In [ ]:
class Graph:
    def __init__(self, num_of_vertices):
        self.v = num_of_vertices
        self.edges = [[-1 for i in range(num_of_vertices)] for j in range(num_of_vertices)]
        self.visited = []
        
    def add_edge(self, u, v, weight):
        self.edges[u][v] = weight


def get_pos(grid, value):
    matches_x, matches_y = np.where(grid == value)
    return np.array([matches_x[0], matches_y[0]])

def get_level(letter):
    if letter == "S":
        return 0
    elif letter == "E":
        return 25
    else:
        return string.ascii_lowercase.index(letter)
    
def can_reach_pos_y_from_pos_x(pos_x, pos_y, grid):
    return (grid[tuple(pos_y)] - grid[tuple(pos_x)]) <= 1
    

def get_pos_in_direction(direction, pos, grid, reverse_direction=False):
    if direction == "up":
        mod = (-1, 0)
    elif direction == "down":
        mod = (1, 0)
    elif direction == "left":
        mod = (0, -1)
    elif direction == "right":
        mod = (0, 1)
    else:
        print("did not recognize")
        
    new_pos = pos + mod
    if (new_pos >= 0).all() and (new_pos < grid.shape).all():
        if not reverse_direction:
            if can_reach_pos_y_from_pos_x(pos_x=pos, pos_y=new_pos, grid=grid):
                return new_pos
        else:
            if can_reach_pos_y_from_pos_x(pos_x=new_pos, pos_y=pos, grid=grid):
                return new_pos
        
    
def convert_xy_to_idx(pos, grid_shape):
    return pos[1] +  pos[0]*grid_shape[1]
    
    
def get_edge_idxs(pos, grid, reverse_direction = False):
    idxs = []
    
    for d in ["up", "down", "left", "right"]:
        edge_pos = get_pos_in_direction(d, pos, grid, reverse_direction)
        if edge_pos is not None:
            idxs.append(convert_xy_to_idx(edge_pos, grid.shape))
            
    return idxs

def dijkstra(edges, start_vertex, num_vertices):
    D = {v:float('inf') for v in range(num_vertices)}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))
    visited = []
    edges = np.copy(edges)

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        visited.append(current_vertex)

        for neighbor in range(num_vertices):
            if edges[current_vertex][neighbor] != -1:
                distance = edges[current_vertex][neighbor]
                if neighbor not in visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance
                    if new_cost < old_cost:
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost
    return D
    
    

In [ ]:
letter_grid = np.array([list(line) for line in data.split("\n")])
start = convert_xy_to_idx(get_pos(letter_grid, "S"), letter_grid.shape)
end = convert_xy_to_idx(get_pos(letter_grid, "E"), letter_grid.shape)
grid = np.array([[get_level(letter) for letter in line] for line in letter_grid])

In [ ]:
g = Graph(grid.shape[0]*grid.shape[1])

for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        pos = np.array([i,j])
        pos_idx = convert_xy_to_idx(pos, grid.shape)
        neighbour_idxs = get_edge_idxs(pos, grid)
        for idx in neighbour_idxs:
            g.add_edge(pos_idx, idx, 1)

result = dijkstra(g.edges, start, g.v)
result[end]


In [ ]:
g = Graph(grid.shape[0]*grid.shape[1])

for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        pos = np.array([i,j])
        pos_idx = convert_xy_to_idx(pos, grid.shape)
        neighbour_idxs = get_edge_idxs(pos, grid, reverse_direction=True)
        for idx in neighbour_idxs:
            g.add_edge(pos_idx, idx, 1)

result = dijkstra(g.edges, end, g.v)
matches_x, matches_y = np.where(grid == 0)
min([result[convert_xy_to_idx([x, y], grid.shape)] for x, y in zip(matches_x, matches_y)])